In [1]:
library(dplyr)
library(readr)
library(tictoc)
library(stringr)
library(tidyr)


Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



In [2]:
mutate_where <- function(.data, condition, ..., envir = parent.frame()) {
  condition <- eval(substitute(condition), .data, envir)
  .data[condition, ] <- .data[condition, ] %>% mutate(...)
  .data
}

In [3]:
matches.raw <- read_csv('results.csv')
dim(matches.raw)

Parsed with column specification:
cols(
  date = col_date(format = ""),
  home_team = col_character(),
  away_team = col_character(),
  home_score = col_integer(),
  away_score = col_integer(),
  tournament = col_character(),
  city = col_character(),
  country = col_character(),
  neutral = col_logical()
)


In [4]:
length(union(matches.raw$home_team,matches.raw$away_team))

[1] 244

In [5]:
foot.teams <- data.frame(team=union(matches.raw$home_team,matches.raw$away_team),elo=1000,match=0)

In [ ]:
matches.selected <- matches.raw

In [13]:
foot.teams <- data.frame(team=union(matches.selected$home_team,matches.selected$away_team),elo=1000,match=0,w=0,l=0,d=0)
tic('elo rating')
for (row in 1:(dim(matches.selected)[1])) {
    current <- matches.selected[row,]
    scoreA <- (sign(current$home_score-current$away_score)+1)/2
    eloA <- foot.teams[foot.teams$team==current$home_team,]$elo
    eloB <- foot.teams[foot.teams$team==current$away_team,]$elo
    nelo <- newelo(eloA,eloB,scoreA)
    #cat(paste(current$home_team,'-',current$away_team,':',eloA,eloB,'->',nelo[1],nelo[2],'\n'))
    foot.teams[foot.teams$team==current$home_team,]$elo <- nelo[1]
    foot.teams[foot.teams$team==current$away_team,]$elo <- nelo[2]
    foot.teams[foot.teams$team==current$home_team,]$match <- foot.teams[foot.teams$team==current$home_team,]$match +1    
    foot.teams[foot.teams$team==current$away_team,]$match <- foot.teams[foot.teams$team==current$away_team,]$match +1
    if (scoreA==1) {
        foot.teams[foot.teams$team==current$home_team,]$w = foot.teams[foot.teams$team==current$home_team,]$w +1
        foot.teams[foot.teams$team==current$away_team,]$l = foot.teams[foot.teams$team==current$away_team,]$l +1
    } else if (scoreA==0) {
        foot.teams[foot.teams$team==current$home_team,]$l = foot.teams[foot.teams$team==current$home_team,]$l +1
        foot.teams[foot.teams$team==current$away_team,]$w = foot.teams[foot.teams$team==current$away_team,]$w +1
    } else {
        foot.teams[foot.teams$team==current$home_team,]$d = foot.teams[foot.teams$team==current$home_team,]$d +1
        foot.teams[foot.teams$team==current$away_team,]$d = foot.teams[foot.teams$team==current$away_team,]$d +1
    }
}
toc()

elo rating: 45.503 sec elapsed


In [15]:
foot.teams$team <- as.character(foot.teams$team)

In [16]:
foot.teams %>% mutate(wpct=round(100*w/match,2),lpct=round(100*l/match,2),dpct=round(100*d/match,2))%>% arrange(desc(elo))

team,elo,match,w,l,d,wpct,lpct,dpct
Iran,1489,184,128,0,56,69.57,0.00,30.43
Germany,1475,248,192,24,32,77.42,9.68,12.90
Portugal,1440,248,176,16,56,70.97,6.45,22.58
Korea Republic,1412,232,152,32,48,65.52,13.79,20.69
Spain,1379,200,160,24,16,80.00,12.00,8.00
England,1378,192,152,8,32,79.17,4.17,16.67
Australia,1375,248,152,32,64,61.29,12.90,25.81
Senegal,1360,232,160,16,56,68.97,6.90,24.14
Japan,1360,240,152,40,48,63.33,16.67,20.00
Brazil,1359,200,120,24,56,60.00,12.00,28.00


### Loop for Stats

In [17]:
newelo <- function(eloA,eloB,scoreA,K=32) {
    EA <- 1/(1+10**((eloB-eloA)/400))
    delta <- round((scoreA-EA)*K,0)
    c(eloA+delta,eloB-delta)
}

In [18]:
probadraw <- function(eloA,eloB) {
    pwin <- 1/(1+10**((eloB-eloA)/400))
    delta <- abs(2*pwin-1)
    (1-delta)*exp(-delta)/2    
}

In [19]:
create_pool_round <- function() {
    countries <- c('Russia','Saudi Arabia','Egypt','Uruguay','Portugal','Spain','Morocco','Iran',
                   'France','Australia','Peru','Denmark','Argentina','Iceland','Croatia','Nigeria',
                   'Brazil','Switzerland','Costa Rica','Serbia','Germany','Mexico','Sweden','Korea Republic',
                   'Belgium','Panama','Tunisia','England','Poland','Senegal','Colombia','Japan' )
    poolteam <- c('A','A','A','A','B','B','B','B',
                  'C','C','C','C','D','D','D','D',
                  'E','E','E','E','F','F','F','F',
                  'G','G','G','G','H','H','H','H')
    pools <- data.frame(team=countries,pool=poolteam,stringsAsFactors=F)
    
    wc.teams <<- foot.teams %>% inner_join(pools,by='team') %>% mutate(match=0,w=0,l=0,d=0)

    date <- c('Jun 14 2018','Jun 15 2018','Jun 15 2018','Jun 15 2018','Jun 16 2018','Jun 16 2018','Jun 16 2018','Jun 16 2018',
              'Jun 17 2018','Jun 17 2018','Jun 17 2018','Jun 18 2018','Jun 18 2018','Jun 18 2018','Jun 19 2018','Jun 19 2018',
              'Jun 19 2018','Jun 20 2018','Jun 20 2018','Jun 20 2018','Jun 21 2018','Jun 21 2018','Jun 21 2018','Jun 22 2018',
              'Jun 22 2018','Jun 22 2018','Jun 23 2018','Jun 23 2018','Jun 23 2018','Jun 24 2018','Jun 24 2018','Jun 24 2018',
              'Jun 25 2018','Jun 25 2018','Jun 25 2018','Jun 25 2018','Jun 26 2018','Jun 26 2018','Jun 26 2018','Jun 26 2018',
              'Jun 27 2018','Jun 27 2018','Jun 27 2018','Jun 27 2018','Jun 28 2018','Jun 28 2018','Jun 28 2018','Jun 28 2018')
    home_team <- c('Russia','Egypt','Portugal','Morocco','France','Peru','Argentina','Croatia',
                   'Brazil','Costa Rica','Germany','Sweden','Belgium','Tunisia','Poland','Colombia',
                   'Russia','Uruguay','Portugal','Iran','France','Denmark','Argentina','Nigeria',
                   'Brazil','Serbia','Germany','Korea Republic','Belgium','England','Poland','Japan',
                   'Uruguay','Saudi Arabia','Iran','Spain','Denmark','Australia','Nigeria','Iceland',
                   'Serbia','Switzerland','Korea Republic','Mexico','England','Panama','Japan','Senegal')
    away_team <- c('Saudi Arabia','Uruguay','Spain','Iran','Australia','Denmark','Iceland','Nigeria',
                   'Switzerland','Serbia','Mexico','Korea Republic','Panama','England','Senegal','Japan',
                   'Egypt','Saudi Arabia','Morocco','Spain','Peru','Australia','Croatia','Iceland',
                   'Costa Rica','Switzerland','Sweden','Mexico','Tunisia','Panama','Colombia','Senegal',
                   'Russia','Egypt','Portugal','Morocco','France','Peru','Argentina','Croatia',
                   'Brazil','Costa Rica','Germany','Sweden','Belgium','Tunisia','Poland','Colombia')
    pool <- c('A','A','B','B','C','C','D','D','E','E','F','F','G','G','H','H',
              'A','A','B','B','C','C','D','D','E','E','F','F','G','G','H','H',
              'A','A','B','B','C','C','D','D','E','E','F','F','G','G','H','H')

    wc.pool <<- data.frame(pool=pool, date=date, home_team=home_team, away_team=away_team, stringsAsFactors=F) %>% mutate(date=as.Date(date,'%b %d %Y'))
}

In [20]:
play_pool_round <- function() {
    for (row in 1:(dim(wc.pool)[1])) {
        home_team_idx = as.numeric(rownames(wc.teams[wc.teams$team==wc.pool[row,]$home_team,]))
        away_team_idx = as.numeric(rownames(wc.teams[wc.teams$team==wc.pool[row,]$away_team,]))

        wc.pool[row,'home_elo'] <- wc.teams[home_team_idx,]$elo
        wc.pool[row,'away_elo'] <- wc.teams[away_team_idx,]$elo
        wc.pool[row,'home_p'] <- 1/(1+10**((wc.pool[row,]$away_elo-wc.pool[row,]$home_elo)/400))
        wc.pool[row,'draw_p'] <- probadraw(wc.pool[row,]$home_elo,wc.pool[row,]$away_elo)
        wc.pool[row,'home_pwin']  <- wc.pool[row,]$home_p - wc.pool[row,]$draw_p/2
        wc.pool[row,'home_pdraw'] <- wc.pool[row,]$home_p + wc.pool[row,]$draw_p/2
        wc.pool[row,'rand'] <- runif(1)

        wc.pool[row,'home_pts'] <- if_else(wc.pool[row,]$rand<=wc.pool[row,]$home_pwin,1,if_else(wc.pool[row,]$rand<=wc.pool[row,]$home_pdraw,0.5,0))

        nelo <- newelo(wc.pool[row,]$home_elo,wc.pool[row,]$away_elo,wc.pool[row,]$home_pts)
        wc.teams[home_team_idx,]$elo <- nelo[1]
        wc.teams[away_team_idx,]$elo <- nelo[2]

        wc.teams[home_team_idx,]$match <- wc.teams[home_team_idx,]$match +1    
        wc.teams[away_team_idx,]$match <- wc.teams[away_team_idx,]$match +1
        if (wc.pool[row,]$home_pts==1) {
            wc.teams[home_team_idx,]$w = wc.teams[home_team_idx,]$w +1
            wc.teams[away_team_idx,]$l = wc.teams[away_team_idx,]$l +1
        } else if (wc.pool[row,]$home_pts==0) {
            wc.teams[home_team_idx,]$l = wc.teams[home_team_idx,]$l +1
            wc.teams[away_team_idx,]$w = wc.teams[away_team_idx,]$w +1
        } else {
            wc.teams[home_team_idx,]$d = wc.teams[home_team_idx,]$d +1
            wc.teams[away_team_idx,]$d = wc.teams[away_team_idx,]$d +1
        }
    }
    wc.teams <<- wc.teams %>% mutate(pts=3*w+d)
    wc.pool <<- wc.pool
}

In [21]:
create_8th_round <- function() {
    wc.qualified <- wc.teams %>% arrange(pool,desc(pts),desc(w),l) %>% group_by(pool) %>% 
                             mutate(pos=row_number()) %>% slice(1:2) %>% mutate(rank=paste0(pool,pos)) %>% ungroup()
    home_team <- c('A1','C1','E1','G1','B1','D1','F1','H1')
    away_team <- c('B2','D2','F2','H2','A2','C2','E2','G2')
    wc.8th <- data.frame(home_pos=home_team,away_pos=away_team,stringsAsFactors=F)
    wc.8th <<- wc.8th %>% inner_join(wc.qualified %>% select(team,rank),by=c('home_pos'='rank')) %>% rename(home_team=team) %>%
                     inner_join(wc.qualified %>% select(team,rank),by=c('away_pos'='rank')) %>% rename(away_team=team)
}

In [22]:
play_8th_round <- function() {
    for (row in 1:(dim(wc.8th)[1])) {
        home_team_idx = as.numeric(rownames(wc.teams[wc.teams$team==wc.8th[row,]$home_team,]))
        away_team_idx = as.numeric(rownames(wc.teams[wc.teams$team==wc.8th[row,]$away_team,]))

        wc.8th[row,'home_elo'] <- wc.teams[home_team_idx,]$elo
        wc.8th[row,'away_elo'] <- wc.teams[away_team_idx,]$elo
        wc.8th[row,'home_p'] <- 1/(1+10**((wc.8th[row,]$away_elo-wc.8th[row,]$home_elo)/400))
        wc.8th[row,'rand'] <- runif(1)
        wc.8th[row,'home_pts'] <- if_else(wc.8th[row,]$rand<=wc.8th[row,]$home_p,1,0)

        wc.teams[home_team_idx,'r8th'] <- wc.8th[row,]$home_pts
        wc.teams[away_team_idx,'r8th'] <- 1-wc.8th[row,]$home_pts

        nelo <- newelo(wc.8th[row,]$home_elo,wc.8th[row,]$away_elo,wc.8th[row,]$home_pts)
        wc.teams[home_team_idx,]$elo <- nelo[1]
        wc.teams[away_team_idx,]$elo <- nelo[2]
    }
    wc.teams <<- wc.teams
    wc.8th <<- wc.8th
}

In [23]:
create_4th_round <- function() {
    home_team <- c(if_else(wc.8th[1,]$home_pts==1,as.character(wc.8th[1,]$home_team),as.character(wc.8th[1,]$away_team)),
                   if_else(wc.8th[3,]$home_pts==1,as.character(wc.8th[3,]$home_team),as.character(wc.8th[3,]$away_team)),
                   if_else(wc.8th[5,]$home_pts==1,as.character(wc.8th[5,]$home_team),as.character(wc.8th[5,]$away_team)),
                   if_else(wc.8th[7,]$home_pts==1,as.character(wc.8th[7,]$home_team),as.character(wc.8th[7,]$away_team)) )

    away_team <- c(if_else(wc.8th[2,]$home_pts==1,as.character(wc.8th[2,]$home_team),as.character(wc.8th[2,]$away_team)),
                   if_else(wc.8th[4,]$home_pts==1,as.character(wc.8th[4,]$home_team),as.character(wc.8th[4,]$away_team)),
                   if_else(wc.8th[6,]$home_pts==1,as.character(wc.8th[6,]$home_team),as.character(wc.8th[6,]$away_team)),
                   if_else(wc.8th[8,]$home_pts==1,as.character(wc.8th[8,]$home_team),as.character(wc.8th[8,]$away_team)) )

    wc.4th <<- data.frame(home_team=home_team,away_team=away_team, stringsAsFactors=F) 
}

In [24]:
play_4th_round <- function() {
    for (row in 1:(dim(wc.4th)[1])) {
        home_team_idx = as.numeric(rownames(wc.teams[wc.teams$team==wc.4th[row,]$home_team,]))
        away_team_idx = as.numeric(rownames(wc.teams[wc.teams$team==wc.4th[row,]$away_team,]))

        wc.4th[row,'home_elo'] <- wc.teams[home_team_idx,]$elo
        wc.4th[row,'away_elo'] <- wc.teams[away_team_idx,]$elo
        wc.4th[row,'home_p'] <- 1/(1+10**((wc.4th[row,]$away_elo-wc.4th[row,]$home_elo)/400))
        wc.4th[row,'rand'] <- runif(1)
        wc.4th[row,'home_pts'] <- if_else(wc.4th[row,]$rand<=wc.4th[row,]$home_p,1,0)

        wc.teams[home_team_idx,'r4th'] <- wc.4th[row,]$home_pts
        wc.teams[away_team_idx,'r4th'] <- 1-wc.4th[row,]$home_pts

        nelo <- newelo(wc.4th[row,]$home_elo,wc.4th[row,]$away_elo,wc.4th[row,]$home_pts)
        wc.teams[home_team_idx,]$elo <- nelo[1]
        wc.teams[away_team_idx,]$elo <- nelo[2]
    }
    wc.teams <<- wc.teams
    wc.4th <<- wc.4th    
}

In [25]:
create_2th_round <- function() {
    home_team <- c(if_else(wc.4th[1,]$home_pts==1,as.character(wc.4th[1,]$home_team),as.character(wc.4th[1,]$away_team)),
                   if_else(wc.4th[3,]$home_pts==1,as.character(wc.4th[3,]$home_team),as.character(wc.4th[3,]$away_team)) )

    away_team <- c(if_else(wc.4th[2,]$home_pts==1,as.character(wc.4th[2,]$home_team),as.character(wc.4th[2,]$away_team)),
                   if_else(wc.4th[4,]$home_pts==1,as.character(wc.4th[4,]$home_team),as.character(wc.4th[4,]$away_team)) )

    wc.2th <<- data.frame(home_team=home_team,away_team=away_team, stringsAsFactors=F)   
}

In [26]:
play_2th_round <- function() {
    for (row in 1:(dim(wc.2th)[1])) {
        home_team_idx = as.numeric(rownames(wc.teams[wc.teams$team==wc.2th[row,]$home_team,]))
        away_team_idx = as.numeric(rownames(wc.teams[wc.teams$team==wc.2th[row,]$away_team,]))

        wc.2th[row,'home_elo'] <- wc.teams[home_team_idx,]$elo
        wc.2th[row,'away_elo'] <- wc.teams[away_team_idx,]$elo
        wc.2th[row,'home_p'] <- 1/(1+10**((wc.2th[row,]$away_elo-wc.2th[row,]$home_elo)/400))
        wc.2th[row,'rand'] <- runif(1)
        wc.2th[row,'home_pts'] <- if_else(wc.2th[row,]$rand<=wc.2th[row,]$home_p,1,0)

        wc.teams[home_team_idx,'r2th'] <- wc.2th[row,]$home_pts
        wc.teams[away_team_idx,'r2th'] <- 1-wc.2th[row,]$home_pts

        nelo <- newelo(wc.2th[row,]$home_elo,wc.2th[row,]$away_elo,wc.2th[row,]$home_pts)
        wc.teams[home_team_idx,]$elo <- nelo[1]
        wc.teams[away_team_idx,]$elo <- nelo[2]
    }
    wc.teams <<- wc.teams
    wc.2th <<- wc.2th
}

In [27]:
create_playoff_round <- function() {
    home_team <- c(if_else(wc.2th[1,]$home_pts==1,as.character(wc.2th[1,]$away_team),as.character(wc.2th[1,]$home_team)))
    away_team <- c(if_else(wc.2th[2,]$home_pts==1,as.character(wc.2th[2,]$away_team),as.character(wc.2th[2,]$home_team)))

    wc.playoff <<- data.frame(home_team=home_team,away_team=away_team, stringsAsFactors=F)    
}

In [28]:
play_playoff_round <- function() {
    home_team_idx = as.numeric(rownames(wc.teams[wc.teams$team==wc.playoff[1,]$home_team,]))
    away_team_idx = as.numeric(rownames(wc.teams[wc.teams$team==wc.playoff[1,]$away_team,]))

    wc.playoff[1,'home_elo'] <- wc.teams[home_team_idx,]$elo
    wc.playoff[1,'away_elo'] <- wc.teams[away_team_idx,]$elo
    wc.playoff[1,'home_p'] <- 1/(1+10**((wc.playoff[1,]$away_elo-wc.playoff[1,]$home_elo)/400))
    wc.playoff[1,'rand'] <- runif(1)
    wc.playoff[1,'home_pts'] <- if_else(wc.playoff[1,]$rand<=wc.playoff[1,]$home_p,1,0)

    wc.teams[home_team_idx,'rplayoff'] <- wc.playoff[1,]$home_pts
    wc.teams[away_team_idx,'rplayoff'] <- 1-wc.playoff[1,]$home_pts

    nelo <- newelo(wc.playoff[1,]$home_elo,wc.playoff[1,]$away_elo,wc.playoff[1,]$home_pts)
    wc.teams[home_team_idx,]$elo <- nelo[1]
    wc.teams[away_team_idx,]$elo <- nelo[2]

    wc.teams <<- wc.teams
    wc.playoff <<- wc.playoff
}

In [29]:
create_final_round <- function() {
    home_team <- c(if_else(wc.2th[1,]$home_pts==1,as.character(wc.2th[1,]$home_team),as.character(wc.2th[1,]$away_team)))
    away_team <- c(if_else(wc.2th[2,]$home_pts==1,as.character(wc.2th[2,]$home_team),as.character(wc.2th[2,]$away_team)))

    wc.final <<- data.frame(home_team=home_team,away_team=away_team, stringsAsFactors=F)   
}

In [30]:
play_final_round <- function() {
    home_team_idx = as.numeric(rownames(wc.teams[wc.teams$team==wc.final[1,]$home_team,]))
    away_team_idx = as.numeric(rownames(wc.teams[wc.teams$team==wc.final[1,]$away_team,]))

    wc.final[1,'home_elo'] <- wc.teams[home_team_idx,]$elo
    wc.final[1,'away_elo'] <- wc.teams[away_team_idx,]$elo
    wc.final[1,'home_p'] <- 1/(1+10**((wc.final[1,]$away_elo-wc.final[1,]$home_elo)/400))
    wc.final[1,'rand'] <- runif(1)
    wc.final[1,'home_pts'] <- if_else(wc.final[1,]$rand<=wc.final[1,]$home_p,1,0)

    wc.teams[home_team_idx,'rfinal'] <- wc.final[1,]$home_pts
    wc.teams[away_team_idx,'rfinal'] <- 1-wc.final[1,]$home_pts

    nelo <- newelo(wc.final[1,]$home_elo,wc.final[1,]$away_elo,wc.final[1,]$home_pts)
    wc.teams[home_team_idx,]$elo <- nelo[1]
    wc.teams[away_team_idx,]$elo <- nelo[2]
    
    wc.teams <<- wc.teams
    wc.final <<- wc.final
}

In [31]:
library(foreach)
library(doParallel)

Loading required package: iterators
Loading required package: parallel


In [32]:
no_cores <- detectCores() - 1  

In [33]:
cl <- makeCluster(no_cores, type="FORK")
registerDoParallel(cl)

tic('Stats')
for(r in 1:100) {
    wc.teams.stats <- foreach(i=1:10000, .combine=rbind) %dopar%
    {
        set.seed(NULL)

        create_pool_round()
        play_pool_round()

        create_8th_round()
        play_8th_round()

        create_4th_round()
        play_4th_round()

        create_2th_round()
        play_2th_round()

        create_playoff_round()
        play_playoff_round()

        create_final_round()
        play_final_round()

        wc.teams.stats <- wc.teams %>% mutate(run=10000*(r-1)+i)
        saveRDS(wc.teams.stats, paste0('wc.teams.stats.',str_pad(r,2,pad='0')))
    }
    
stopCluster(cl)
toc()
    
dim(wc.teams.stats)

Stats: 120.296 sec elapsed


[1] 32000    14

In [ ]:
wc.teams.stats.all <- readRDS('wc.teams.stats.01')
for(r in 2:100) {
    wc.teams.stats <- readRDS('wc.teams.stats.', paste0('wc.teams.stats.',str_pad(r,2,pad='0')))
    wc.teams.stats.all <- wc.teams.stats.all %>% rbind(wc.teams.stats)
    }

In [34]:
wc.teams.stats %>% select(team,r8th,r4th,r2th,rplayoff,rfinal) %>% 
                   replace_na(list(r8th=0,r4th=0,r2th=0,rplayoff=0,rfinal=0)) %>% 
                   group_by(team) %>% summarise_all(funs(mean(.,na.rm=T))) %>%
                   arrange(desc(rfinal),desc(rplayoff),desc(r2th),desc(r4th),desc(r8th))

team,r8th,r4th,r2th,rplayoff,rfinal
Iran,0.561,0.403,0.259,0.103,0.172
Germany,0.612,0.421,0.270,0.102,0.169
Portugal,0.574,0.391,0.221,0.099,0.118
Korea Republic,0.448,0.269,0.152,0.062,0.071
Australia,0.542,0.298,0.138,0.084,0.063
Spain,0.321,0.173,0.090,0.042,0.049
England,0.435,0.216,0.102,0.059,0.045
Senegal,0.414,0.192,0.103,0.040,0.042
Brazil,0.310,0.163,0.079,0.045,0.037
Japan,0.306,0.154,0.072,0.048,0.033


In [35]:
wc.teams.stats %>% filter(rfinal==1) %>% group_by(team) %>% summarize(n1=n()) %>% arrange(desc(n1)) %>% head(5)
predict.first <- wc.teams.stats %>% filter(rfinal==1) %>% group_by(team) %>% summarize(n1=n()) %>% arrange(desc(n1)) %>% head(1)

team,n1
Iran,172
Germany,169
Portugal,118
Korea Republic,71
Australia,63


In [36]:
wc.teams.stats %>% filter(rfinal==0) %>% anti_join(predict.first,by='team') %>% group_by(team) %>% summarize(n2=n()) %>% arrange(desc(n2)) %>% head(5)
predict.second <- wc.teams.stats %>% filter(rfinal==0) %>% anti_join(predict.first,by='team') %>% 
        group_by(team) %>% summarize(n2=n()) %>% arrange(desc(n2)) %>% head(1)

team,n2
Portugal,103
Germany,101
Korea Republic,81
Australia,75
Senegal,61


In [37]:
wc.teams.stats %>% filter(rplayoff==1) %>% anti_join(predict.first,by='team') %>% anti_join(predict.second,by='team') %>%
        group_by(team) %>% summarize(n3=n()) %>% arrange(desc(n3)) %>% head(5)
predict.third <- wc.teams.stats %>% filter(rplayoff==1) %>% anti_join(predict.first,by='team') %>% anti_join(predict.second,by='team') %>% 
        group_by(team) %>% summarize(n3=n()) %>% arrange(desc(n3)) %>% head(1)

team,n3
Germany,102
Australia,84
Korea Republic,62
England,59
Japan,48


In [38]:
wc.teams.stats %>% filter(rplayoff==0) %>% anti_join(predict.first,by='team') %>% anti_join(predict.second,by='team') %>% anti_join(predict.third,by='team') %>%
        group_by(team) %>% summarize(n4=n()) %>% arrange(desc(n4)) %>% head(5)
predict.four <- wc.teams.stats %>% filter(rplayoff==0) %>% anti_join(predict.first,by='team') %>% anti_join(predict.second,by='team') %>% anti_join(predict.third,by='team') %>%
        group_by(team) %>% summarize(n4=n()) %>% arrange(desc(n4)) %>% head(1)

team,n4
Australia,76
England,55
Korea Republic,55
Poland,49
Senegal,49


In [39]:
predict.first
predict.second
predict.third
predict.four

team,n1
Iran,172


team,n2
Portugal,103


team,n3
Germany,102


team,n4
Australia,76


In [41]:
tmp01 <- wc.teams.stats %>% filter(rfinal>=0) %>% select(team,run,rfinal) %>% spread(rfinal,team) %>% rename(t1='0',t2='1') %>%
                group_by(t1,t2) %>% summarize(n=n()) %>% ungroup()
tmp02 <- tmp01 %>% inner_join(tmp01,by=c('t1'='t2','t2'='t1')) %>% mutate(total=n.x+n.y)
tmp02 %>% arrange(desc(total)) %>% head()

t1,t2,n.x,n.y,total
Iran,Portugal,17,34,51
Portugal,Iran,34,17,51
Germany,Iran,20,17,37
Germany,Korea Republic,15,22,37
Iran,Germany,17,20,37
Korea Republic,Germany,22,15,37


In [42]:
tmp01 <- wc.teams.stats %>% filter(rplayoff>=0) %>% select(team,run,rplayoff) %>% spread(rplayoff,team) %>% rename(t1='0',t2='1') %>%
                group_by(t1,t2) %>% summarize(n=n()) %>% ungroup()
tmp02 <- tmp01 %>% inner_join(tmp01,by=c('t1'='t2','t2'='t1')) %>% mutate(total=n.x+n.y)
tmp02 %>% arrange(desc(total)) %>% head()

t1,t2,n.x,n.y,total
Iran,Portugal,8,14,22
Portugal,Iran,14,8,22
Germany,Portugal,8,13,21
Portugal,Germany,13,8,21
Australia,Germany,10,7,17
Germany,Australia,7,10,17


In [44]:
tmp02 %>% filter(!t1 %in% c('Iran','Portugal')) %>% filter(!t2 %in% c('Iran','Portugal')) %>% arrange(desc(total)) %>% head()

t1,t2,n.x,n.y,total
Australia,Germany,10,7,17
Germany,Australia,7,10,17
Australia,France,3,9,12
Australia,Senegal,7,5,12
France,Australia,9,3,12
Germany,Poland,3,9,12
